In [0]:
# Notebook 2: SILVER - Upsert incremental

from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from delta.tables import DeltaTable

spark = SparkSession.builder.getOrCreate()

BRONZE_TABLE = "workspace.javier_altoe.crypto_bronze"
SILVER_TABLE = "workspace.javier_altoe.crypto_silver"

df_bronze = spark.table(BRONZE_TABLE)

# Selección de columnas
df_silver_new = df_bronze.select(
    col("id"),
    col("name"),
    col("symbol"),
    col("quote.USD.price").alias("price_usd"),
    col("quote.USD.market_cap").alias("market_cap"),
    col("quote.USD.volume_24h").alias("volume_24h"),
    col("last_updated")
)

# Si la tabla Silver no existe, la creamos
if not spark._jsparkSession.catalog().tableExists(SILVER_TABLE):
    df_silver_new.write.format("delta").saveAsTable(SILVER_TABLE)
else:
    delta_silver = DeltaTable.forName(spark, SILVER_TABLE)
    delta_silver.alias("t").merge(
        df_silver_new.alias("s"),
        "t.id = s.id"
    ).whenMatchedUpdateAll(
    ).whenNotMatchedInsertAll(
    ).execute()

print("✅ Silver actualizado incrementalmente")
